In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class LuiScrapper():
    #NOTE: private params?
    def __init__(self, user, password,
        options=["--headless", "--no-sandbox", "--disable-dev-shm-usage"]):
        self.user = user
        self.password = password
        self.options = options
        self.sites = None
        #self.sites = {"277434":"dev.alza.sandbox","239969":"alza.hu",
        #    "240068":"alza.at","240122":"alza.de", "292410":"dev.alza.at",
        #    "239813":"alza.cz", "239900":"alza.sk","271597":"alza.fr",
        #    "271649":"alza.co.uk","271658":"alzashop.com"}
        self.raw_tabs = {}
        self.df = None
    
    def connect(self):
        options = Options()
        for o in self.options:
            options.add_argument(o)
        self.driver = webdriver.Chrome(service=\
            Service(ChromeDriverManager().install()), options=options)
        return self
    
    def login(self):
        self.driver.get("https://app.luigisbox.com/users/sign_in")
        username = self.driver.find_element("id","user_email")
        username.clear()
        username.send_keys(self.user)
        passw = self.driver.find_element("id","user_password")
        passw.clear()
        passw.send_keys(self.password)
        self.driver.find_element("name", "commit").click()
        return self

    def get_tables(self):
        if self.sites is None:
            self._get_sites()
        for k,v in self.sites.items():
            self.driver.get(f"https://app.luigisbox.com/sites/208957-239813/settings/unit_stats?sites={k}")
            self.raw_tabs[v] = self.driver.find_element(By.CLASS_NAME,"table").get_attribute('outerHTML')
        return self
    
    def parse_tables(self):
        if len(self.raw_tabs)==0:
            return None
        self.df = pd.concat([self._parse_table(k,v)
            for k,v in self.raw_tabs.items()])
        return self

    def close(self):
        self.driver.close()
        return self
    
    def _get_sites(self):
        self.driver.get(f"https://app.luigisbox.com/sites/208957-239813/settings/unit_stats")
        sites = self.driver.find_elements(By.CLASS_NAME,"fancy-checkbox--pill-with-top-margin")
        self.sites = {s.find_element(By.CLASS_NAME,"fancy-checkbox__input").get_attribute("value"):\
                      s.get_attribute("title").replace(" ","") for s in sites}
        return self
    
    def _parse_table(self,k,v):
        # NOTE: fix column formatting
        tab = pd.read_html(v)[0]
        cols = [i[0].lower().replace(" ","_") for i in tab.columns]
        tab = tab.T.reset_index(drop=True).T
        tab.columns = cols
        tab.loc[:,"country"] = k
        tab.loc[:,"timestamp"] = pd.to_datetime(datetime.now())
        tab = pd.melt(tab, id_vars=["metric", "country", "timestamp"],
            var_name="month_year")
        tab["month_year"] = tab.apply(self._map_dates, axis=1)
        return tab

    def _map_dates(self, row):
        if row["month_year"]=="current_month":
            my = row["timestamp"].strftime("%m%Y")
        elif row["month_year"]=="previous_month":
            my = (row["timestamp"].replace(day=1)-timedelta(days=1))\
                .strftime("%m%Y")
        else:
            my = np.nan
        return my

s = LuiScrapper(user="", password="")
s.connect().login().get_tables().parse_tables().close()
s.df[s.df.country=="alza.at"]         

,metric,country,timestamp,month_year,value
0,Pageviews,alza.at,2023-04-25 09:05:14.843512,032023,4893593
1,Max. number of records in catalog,alza.at,2023-04-25 09:05:14.843512,032023,355216
2,Searches,alza.at,2023-04-25 09:05:14.843512,032023,76779
3,Autocompletes,alza.at,2023-04-25 09:05:14.843512,032023,157043
4,Recommenders,alza.at,2023-04-25 09:05:14.843512,032023,0
5,Banners impressions,alza.at,2023-04-25 09:05:14.843512,032023,0
6,Product listing,alza.at,2023-04-25 09:05:14.843512,032023,0
7,Total units,alza.at,2023-04-25 09:05:14.843512,032023,708084
8,Voice searches,alza.at,2023-04-25 09:05:14.843512,032023,0
9,Catalog updates,alza.at,2023-04-25 09:05:14.843512,032023,83246
